In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_curve, auc
import warnings
import os
from datetime import datetime

warnings.filterwarnings("ignore")

In [3]:
# Load Data

def load_data(file_path):
    CHUNKSIZE = 100000
    # Removing 'ts': 'int64' to allow pandas to infer the type for ts
    dtypes = {'userId': 'category', 'page': 'category'}
    return pd.concat(pd.read_json(file_path, lines=True, chunksize=CHUNKSIZE, dtype=dtypes))

In [4]:
# Exploratory Data Analysis

def perform_eda(df):
    print("\n===== EXPLORATORY DATA ANALYSIS =====")
    print(f"Dataset shape: {df.shape}")
    print("\nData types:")
    print(df.dtypes)

    print("\nMissing values per column:")
    print(df.isnull().sum())

    print("\nBasic statistics:")
    print(df.describe())

    print("\nUnique values in categorical columns:")
    for col in df.select_dtypes(include=['category', 'object']).columns:
        print(f"{col}: {df[col].nunique()} unique values")

    print("\nPage distribution:")
    page_counts = df['page'].value_counts()
    print(page_counts)

    # Create EDA visualizations
    os.makedirs("plots/eda", exist_ok=True)

    # Plot page distribution
    plt.figure(figsize=(12, 6))
    page_counts.head(15).plot(kind='bar')
    plt.title('Top 15 Pages by Frequency')
    plt.ylabel('Count')
    plt.xlabel('Page')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig("plots/eda/page_distribution.png")
    plt.close()

    # Plot user levels
    plt.figure(figsize=(8, 5))
    df['level'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title('User Subscription Level Distribution')
    plt.ylabel('')
    plt.tight_layout()
    plt.savefig("plots/eda/user_levels.png")
    plt.close()

    # Plot gender distribution
    plt.figure(figsize=(8, 5))
    df['gender'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title('Gender Distribution')
    plt.ylabel('')
    plt.tight_layout()
    plt.savefig("plots/eda/gender_distribution.png")
    plt.close()

    # Convert 'ts' to datetime before time analysis
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')

    # Time analysis
    df['hour'] = df['ts'].dt.hour

    plt.figure(figsize=(12, 6))
    df['hour'].value_counts().sort_index().plot(kind='bar')
    plt.title('User Activity by Hour of Day')
    plt.xlabel('Hour of Day')
    plt.ylabel('Number of Actions')
    plt.tight_layout()
    plt.savefig("plots/eda/activity_by_hour.png")
    plt.close()

    # User agent analysis
    if 'userAgent' in df.columns:
        # Extract device type from user agent
        def get_device_type(user_agent):
            if pd.isna(user_agent):
                return "Unknown"
            ua = user_agent.lower()
            if 'iphone' in ua or 'ipad' in ua or 'ios' in ua:
                return "iOS"
            elif 'android' in ua:
                return "Android"
            elif 'windows' in ua or 'mac' in ua or 'linux' in ua:
                return "Desktop"
            else:
                return "Other"

        df['device_type'] = df['userAgent'].apply(get_device_type)

        plt.figure(figsize=(10, 6))
        df['device_type'].value_counts().plot(kind='bar')
        plt.title('Device Type Distribution')
        plt.xlabel('Device Type')
        plt.ylabel('Count')
        plt.tight_layout()
        plt.savefig("plots/eda/device_distribution.png")
        plt.close()

    return df

In [5]:
# Preprocess and Feature Engineering

def preprocess_data(df):
    print("\nPreprocessing & Feature Engineering...")
    df = df[df['userId'] != '']
    df['churn'] = (df['page'] == 'Cancellation Confirmation').astype(int)
    df['ts'] = pd.to_datetime(df['ts'], unit='ms')
    df['registration'] = pd.to_datetime(df['registration'], unit='ms')
    df['session_duration'] = df.groupby(['userId', 'sessionId'])['ts'].transform(lambda x: (x.max() - x.min()).total_seconds())

    # Enhanced feature engineering
    if 'page' in df.columns:
        df['is_home'] = (df['page'] == 'Home').astype(int)
        df['is_settings'] = (df['page'] == 'Settings').astype(int)
        df['is_help'] = (df['page'] == 'Help').astype(int)
        df['is_downgrade'] = (df['page'] == 'Downgrade').astype(int)
        df['is_upgrade'] = (df['page'] == 'Upgrade').astype(int)
        df['is_about'] = (df['page'] == 'About').astype(int)
        df['is_next_song'] = (df['page'] == 'NextSong').astype(int)
        df['is_thumbs_up'] = (df['page'] == 'Thumbs Up').astype(int)
        df['is_thumbs_down'] = (df['page'] == 'Thumbs Down').astype(int)
        df['is_error'] = (df['page'] == 'Error').astype(int)

    current_time = df['ts'].max()

    agg_funcs = {
        'length': ['sum', 'mean', 'count', 'max', 'std'],
        'sessionId': 'nunique',
        'ts': [
            lambda x: (x.max() - x.min()).total_seconds(),
            lambda x: (current_time - x.max()).total_seconds()
        ],
        'session_duration': ['mean', 'max', 'std'],
        'churn': 'max',
        'itemInSession': ['max', 'mean', 'std'],
        'page': 'nunique',
        'artist': 'nunique',
        'song': 'nunique',
        'userAgent': 'nunique',
        'method': 'nunique'
    }

    for col in ['is_home', 'is_settings', 'is_help', 'is_downgrade', 'is_upgrade',
                'is_about', 'is_next_song', 'is_thumbs_up', 'is_thumbs_down', 'is_error']:
        if col in df.columns:
            agg_funcs[col] = 'sum'

    user_features = df.groupby('userId').agg(agg_funcs)
    user_features.columns = ['_'.join([col[0], str(col[1])]) if isinstance(col, tuple) else col
                              for col in user_features.columns]

    column_mapping = {
        'length_sum': 'total_listen',
        'length_mean': 'avg_listen',
        'length_count': 'listen_count',
        'length_max': 'max_listen',
        'length_std': 'std_listen',
        'sessionId_nunique': 'sessions',
        'ts_<lambda_0>': 'active_duration',
        'ts_<lambda_1>': 'recency',
        'session_duration_mean': 'avg_session_duration',
        'session_duration_max': 'max_session_duration',
        'session_duration_std': 'std_session_duration',
        'churn_max': 'churn',
        'itemInSession_max': 'max_item_in_session',
        'itemInSession_mean': 'avg_item_in_session',
        'itemInSession_std': 'std_item_in_session',
        'page_nunique': 'unique_pages',
        'artist_nunique': 'unique_artists',
        'song_nunique': 'unique_songs',
        'userAgent_nunique': 'unique_agents',
        'method_nunique': 'unique_methods'
    }

    user_features = user_features.rename(columns=column_mapping)
    user_features = user_features.reset_index()

    last_levels = df.sort_values('ts').groupby('userId')['level'].last()
    genders = df.groupby('userId')['gender'].first()
    user_features['is_paid'] = (last_levels == 'paid').astype(int)
    user_features['is_male'] = (genders == 'M').astype(int)

    reg_dates = df.groupby('userId')['registration'].first()
    last_ts = df.groupby('userId')['ts'].max()
    user_features['membership_days'] = (last_ts - reg_dates).dt.total_seconds() / (3600 * 24)

    if 'is_next_song_sum' in user_features.columns:
        user_features['next_song_ratio'] = user_features['is_next_song_sum'] / user_features['listen_count']

    if 'is_thumbs_up_sum' in user_features.columns and 'is_thumbs_down_sum' in user_features.columns:
        user_features['satisfaction_ratio'] = user_features['is_thumbs_up_sum'] / (user_features['is_thumbs_down_sum'] + 1)

    if 'is_error_sum' in user_features.columns:
        user_features['error_rate'] = user_features['is_error_sum'] / user_features['listen_count']

    user_features['artist_diversity'] = user_features['unique_artists'] / user_features['listen_count']
    user_features['song_diversity'] = user_features['unique_songs'] / user_features['listen_count']
    user_features['usage_frequency'] = user_features['sessions'] / (user_features['membership_days'] + 1)
    user_features['avg_session_items'] = user_features['listen_count'] / (user_features['sessions'] + 1)

    user_features = user_features.fillna(0).replace([np.inf, -np.inf], 0)

    print(f"Created {len(user_features.columns)} features for {len(user_features)} users")
    return user_features


In [6]:
# Clustering

def cluster_users(user_features):
    print("\nClustering users into behavioral segments...")
    cluster_cols = [
        'total_listen', 'avg_listen', 'sessions', 'active_duration',
        'membership_days', 'unique_artists', 'unique_songs',
        'avg_session_duration', 'is_paid'
    ]
    cluster_cols = [col for col in cluster_cols if col in user_features.columns]
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(user_features[cluster_cols])
    n_components_range = range(2, 11)
    bic_scores = []

    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components, random_state=42)
        gmm.fit(X_scaled)
        bic_scores.append(gmm.bic(X_scaled))

    plt.figure(figsize=(10, 6))
    plt.plot(n_components_range, bic_scores, marker='o')
    plt.xlabel('Number of Components')
    plt.ylabel('BIC Score')
    plt.title('BIC Score by Number of Components')
    plt.grid(True)
    plt.tight_layout()
    os.makedirs("plots", exist_ok=True)
    plt.savefig("plots/bic_score.png")
    plt.close()

    optimal_n_components = n_components_range[np.argmin(bic_scores)]
    print(f"Optimal number of clusters: {optimal_n_components}")
    gmm = GaussianMixture(n_components=optimal_n_components, random_state=42)
    user_features['cluster'] = gmm.fit_predict(X_scaled)

    cluster_analysis = user_features.groupby('cluster').agg({
        'churn': 'mean',
        'total_listen': 'mean',
        'avg_listen': 'mean',
        'sessions': 'mean',
        'active_duration': 'mean',
        'membership_days': 'mean',
        'is_paid': 'mean',
        'unique_artists': 'mean',
        'unique_songs': 'mean',
        'avg_session_duration': 'mean',
        'userId': 'count'
    }).rename(columns={'userId': 'count'})

    cluster_analysis = cluster_analysis.sort_values('churn', ascending=False)
    print("\n===== CLUSTER ANALYSIS =====")
    print(cluster_analysis)
    cluster_analysis.to_csv("cluster_analysis.csv")

    plt.figure(figsize=(12, 8))
    sns.heatmap(cluster_analysis.drop('count', axis=1), cmap='coolwarm', annot=True, fmt=".2f")
    plt.title('Cluster Characteristics')
    plt.tight_layout()
    plt.savefig("plots/cluster_characteristics.png")
    plt.close()

    interpret_clusters(cluster_analysis)
    return user_features

In [7]:
# Cluster Interpretation

def interpret_clusters(cluster_analysis):
    print("\n===== CLUSTER INTERPRETATION =====")
    cluster_interpretations = {}
    risk_levels = {}

    for cluster in cluster_analysis.index:
        data = cluster_analysis.loc[cluster]
        churn_rate = data['churn']
        if churn_rate >= 0.4:
            risk_level = "High Risk"
        elif churn_rate >= 0.2:
            risk_level = "Medium Risk"
        else:
            risk_level = "Low Risk"
        risk_levels[cluster] = risk_level

        is_paid = "Paid" if data['is_paid'] > 0.5 else "Free"

        if data['active_duration'] > cluster_analysis['active_duration'].median():
            activity = "High activity"
        else:
            activity = "Low activity"

        if data['membership_days'] > cluster_analysis['membership_days'].median():
            tenure = "Long-term"
        else:
            tenure = "New"

        if data['unique_artists'] > cluster_analysis['unique_artists'].median():
            diversity = "Diverse taste"
        else:
            diversity = "Focused taste"

        description = f"{risk_level} - {is_paid} users with {activity}, {tenure}, {diversity}"
        recommendations = []
        if risk_level == "High Risk":
            if is_paid == "Paid":
                recommendations.append("Offer discounts or premium features")
            else:
                recommendations.append("Engage with targeted content recommendations")
            if activity == "Low activity":
                recommendations.append("Send re-engagement emails with personalized playlists")
            if tenure == "New":
                recommendations.append("Improve onboarding experience")
            else:
                recommendations.append("Highlight long-term benefits and loyalty rewards")
        elif risk_level == "Medium Risk":
            if diversity == "Focused taste":
                recommendations.append("Recommend new artists similar to their favorites")
            else:
                recommendations.append("Create personalized discovery playlists")
            if is_paid == "Free":
                recommendations.append("Highlight premium benefits")
        else:
            recommendations.append("Maintain engagement with new features")
            recommendations.append("Leverage these users for referrals")

        cluster_interpretations[cluster] = {
            'description': description,
            'size': data['count'],
            'churn_rate': f"{churn_rate:.2%}",
            'recommendations': recommendations
        }

    for cluster, interp in cluster_interpretations.items():
        print(f"\nCluster {cluster}: {interp['description']}")
        print(f"Size: {interp['size']} users")
        print(f"Churn Rate: {interp['churn_rate']}")
        print("Recommendations:")
        for i, rec in enumerate(interp['recommendations'], 1):
            print(f"  {i}. {rec}")

    with open("cluster_interpretations.txt", "w") as f:
        f.write("===== CLUSTER INTERPRETATIONS =====\n\n")
        for cluster, interp in cluster_interpretations.items():
            f.write(f"Cluster {cluster}: {interp['description']}\n")
            f.write(f"Size: {interp['size']} users\n")
            f.write(f"Churn Rate: {interp['churn_rate']}\n")
            f.write("Recommendations:\n")
            for i, rec in enumerate(interp['recommendations'], 1):
                f.write(f"  {i}. {rec}\n")
            f.write("\n")
    return cluster_interpretations, risk_levels

In [8]:
# Visualization

def visualize_data(df):
    print("\nGenerating visualizations...")
    os.makedirs("plots", exist_ok=True)

    plt.figure(figsize=(6, 4))
    sns.countplot(data=df, x='churn')
    plt.title("Churn Distribution")
    plt.xticks([0, 1], ['Not Churned', 'Churned'])
    plt.ylabel("Number of Users")
    plt.tight_layout()
    plt.savefig("plots/churn_distribution.png")
    plt.close()

    for col in ['total_listen', 'avg_listen', 'sessions', 'active_duration', 'membership_days']:
        plt.figure(figsize=(6, 4))
        sns.boxplot(data=df, x='churn', y=col)
        plt.title(f"{col} vs Churn")
        plt.xticks([0, 1], ['Not Churned', 'Churned'])
        plt.tight_layout()
        plt.savefig(f"plots/{col}_vs_churn.png")
        plt.close()

    corr = df.drop(columns=['userId']).corr()
    plt.figure(figsize=(12, 10))
    sns.heatmap(corr.iloc[:20, :20], annot=True, fmt=".2f", cmap='coolwarm', center=0)
    plt.title("Top Feature Correlation Heatmap")
    plt.tight_layout()
    plt.savefig("plots/correlation_heatmap.png")
    plt.close()

    if 'cluster' in df.columns:
        cluster_churn = df.groupby('cluster')['churn'].mean().reset_index()
        plt.figure(figsize=(8, 5))
        sns.barplot(data=cluster_churn, x='cluster', y='churn')
        plt.title("Churn Rate by Cluster")
        plt.ylabel("Churn Rate")
        plt.xlabel("Cluster")
        plt.tight_layout()
        plt.savefig("plots/cluster_vs_churn.png")
        plt.close()

        plt.figure(figsize=(10, 6))
        cluster_counts = df['cluster'].value_counts().sort_index()
        ax = cluster_counts.plot(kind='bar')
        plt.title("Number of Users by Cluster")
        plt.xlabel("Cluster")
        plt.ylabel("Number of Users")
        total = cluster_counts.sum()
        for i, count in enumerate(cluster_counts):
            percentage = 100 * count / total
            ax.annotate(f"{percentage:.1f}%", (i, count), ha='center', va='bottom')
        plt.tight_layout()
        plt.savefig("plots/cluster_distribution.png")
        plt.close()

        for feature in ['total_listen', 'avg_listen', 'membership_days', 'is_paid']:
            plt.figure(figsize=(10, 6))
            sns.boxplot(data=df, x='cluster', y=feature)
            plt.title(f"{feature} Distribution by Cluster")
            plt.tight_layout()
            plt.savefig(f"plots/cluster_{feature}.png")
            plt.close()

In [9]:
# Hyperparameter Tuning

def tune_hyperparameters(X_train, y_train):
    print("\n===== HYPERPARAMETER TUNING =====")

    # Logistic Regression
    print("\nTuning Logistic Regression...")
    lr_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('model', LogisticRegression(max_iter=1000))
    ])

    lr_param_grid = {
        'model__C': [0.001, 0.01, 0.1, 1, 10, 100],
        'model__class_weight': ['balanced', None],
        'model__solver': ['liblinear', 'saga']
    }

    lr_grid = GridSearchCV(lr_pipeline, lr_param_grid, cv=3, scoring='roc_auc')
    lr_grid.fit(X_train, y_train)
    print(f"Best Logistic Regression parameters: {lr_grid.best_params_}")
    print(f"Best ROC AUC score: {lr_grid.best_score_:.3f}")

    # Random Forest
    print("\nTuning Random Forest...")
    rf_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('model', RandomForestClassifier())
    ])

    rf_param_grid = {
        'model__n_estimators': [50, 100, 200],
        'model__max_depth': [None, 10, 20],
        'model__min_samples_split': [2, 5, 10],
        'model__class_weight': ['balanced', 'balanced_subsample', None]
    }

    rf_grid = GridSearchCV(rf_pipeline, rf_param_grid, cv=3, scoring='roc_auc')
    rf_grid.fit(X_train, y_train)
    print(f"Best Random Forest parameters: {rf_grid.best_params_}")
    print(f"Best ROC AUC score: {rf_grid.best_score_:.3f}")

    # XGBoost
    print("\nTuning XGBoost...")
    xgb_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('model', XGBClassifier(use_label_encoder=False, eval_metric='logloss'))
    ])

    xgb_param_grid = {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.3],
        'model__max_depth': [3, 5, 7],
        'model__scale_pos_weight': [1, 5, 25]
    }

    xgb_grid = GridSearchCV(xgb_pipeline, xgb_param_grid, cv=3, scoring='roc_auc')
    xgb_grid.fit(X_train, y_train)
    print(f"Best XGBoost parameters: {xgb_grid.best_params_}")
    print(f"Best ROC AUC score: {xgb_grid.best_score_:.3f}")

    # LightGBM with verbosity fixed
    print("\nTuning LightGBM...")
    lgb_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='median')),
        ('model', lgb.LGBMClassifier(verbosity=-1))  # Suppress LightGBM warnings
    ])

    lgb_param_grid = {
        'model__n_estimators': [50, 100, 200],
        'model__learning_rate': [0.01, 0.1, 0.3],
        'model__num_leaves': [31, 50, 100],
        'model__class_weight': ['balanced', None]
    }

    lgb_grid = GridSearchCV(lgb_pipeline, lgb_param_grid, cv=3, scoring='roc_auc')
    lgb_grid.fit(X_train, y_train)
    print(f"Best LightGBM parameters: {lgb_grid.best_params_}")
    print(f"Best ROC AUC score: {lgb_grid.best_score_:.3f}")

    best_models = {
        'Logistic Regression': lr_grid.best_estimator_,
        'Random Forest': rf_grid.best_estimator_,
        'XGBoost': xgb_grid.best_estimator_,
        'LightGBM': lgb_grid.best_estimator_
    }
    return best_models

In [10]:
# Train model

def train_models(X, y, tuned_models=None):
    if tuned_models:
        print("\nUsing tuned models...")
        return tuned_models

    print("\nTraining models with default parameters...")
    models = {
        'Logistic Regression': Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('model', LogisticRegression(class_weight='balanced', max_iter=1000))
        ]),
        'Random Forest': Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('model', RandomForestClassifier(class_weight='balanced'))
        ]),
        'XGBoost': Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('model', XGBClassifier(scale_pos_weight=25, use_label_encoder=False, eval_metric='logloss'))
        ]),
        'LightGBM': Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            # Set verbosity=-1 to suppress warning messages
            ('model', lgb.LGBMClassifier(class_weight='balanced', verbosity=-1))
        ])
    }

    trained = {}
    for name, model in models.items():
        model.fit(X, y)
        trained[name] = model
    return trained

In [11]:
# Feature Importance

def get_top_features(models, feature_names, top_n=5):
    xgb_imp = models['XGBoost'].named_steps['model'].feature_importances_
    lgbm_imp = models['LightGBM'].named_steps['model'].feature_importances_
    df_imp = pd.DataFrame({
        'Feature': feature_names,
        'XGBoost': xgb_imp,
        'LightGBM': lgbm_imp
    })
    df_imp['Average'] = df_imp[['XGBoost', 'LightGBM']].mean(axis=1)

    plt.figure(figsize=(12, 8))
    top_features_df = df_imp.sort_values('Average', ascending=False).head(15)
    plt.barh(top_features_df['Feature'], top_features_df['Average'])
    plt.xlabel('Average Importance')
    plt.ylabel('Feature')
    plt.title('Top 15 Features by Importance')
    plt.tight_layout()
    plt.savefig("plots/feature_importance.png")
    plt.close()

    return df_imp.sort_values('Average', ascending=False).head(top_n)['Feature'].tolist()

In [12]:
# Evaluate model

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    print(classification_report(y_test, y_pred))
    print(f"ROC AUC: {roc_auc_score(y_test, y_proba):.3f}")
    return {
        'roc_auc': roc_auc_score(y_test, y_proba),
        'classification_report': classification_report(y_test, y_pred, output_dict=True)
    }

def plot_roc_curves(models, X_test, y_test):
    plt.figure(figsize=(8, 6))
    for name, model in models.items():
        y_proba = model.predict_proba(X_test)[:, 1]
        fpr, tpr, _ = roc_curve(y_test, y_proba)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.2f})")
    plt.plot([0, 1], [0, 1], linestyle='--', color='gray')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve Comparison")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    os.makedirs("plots", exist_ok=True)
    plt.savefig("plots/roc_curves.png")
    plt.close()
    return {name: auc(roc_curve(y_test, model.predict_proba(X_test)[:, 1])[0],
                     roc_curve(y_test, model.predict_proba(X_test)[:, 1])[1])
            for name, model in models.items()}

In [13]:
# Business Recommendations
def generate_business_recommendations(models, feature_names, evaluation_metrics, cluster_interpretations=None):
    print("\n===== BUSINESS RECOMMENDATIONS =====")
    best_model_name = max(evaluation_metrics, key=lambda x: evaluation_metrics[x])
    best_auc = evaluation_metrics[best_model_name]
    print(f"The best performing model is {best_model_name} with ROC AUC of {best_auc:.3f}")

    if best_model_name in ['XGBoost', 'LightGBM']:
        model_importances = models[best_model_name].named_steps['model'].feature_importances_
        feature_importance = pd.DataFrame({
            'Feature': feature_names,
            'Importance': model_importances
        }).sort_values('Importance', ascending=False)
        top_10_features = feature_importance.head(10)
        print("\nTop 10 Factors Influencing Churn:")
        for i, (feature, importance) in enumerate(zip(top_10_features['Feature'], top_10_features['Importance']), 1):
            print(f"{i}. {feature}: {importance:.4f}")

    recommendations = []
    recommendations.append({
        'category': 'Strategic',
        'title': 'Implement Predictive Churn Prevention Program',
        'description': f'Deploy the {best_model_name} model as part of a proactive customer retention system. This model achieves {best_auc:.1%} AUC and can identify at-risk users before they churn.',
        'implementation': 'Create an automated alert system that flags users with high churn probability for immediate retention interventions.'
    })

    if 'membership_days' in feature_names:
        recommendations.append({
            'category': 'New Users',
            'title': 'Enhanced Onboarding Experience',
            'description': 'Users with low membership days are at higher risk of churning. Focus on improving the first 30 days of user experience.',
            'implementation': 'Create a structured onboarding program with guided tutorials, personalized recommendations, and early engagement incentives.'
        })

    if 'total_listen' in feature_names or 'avg_listen' in feature_names:
        recommendations.append({
            'category': 'Engagement',
            'title': 'Listening Activity Incentives',
            'description': 'Users with lower listening activity are more likely to churn. Increase engagement through personalized content.',
            'implementation': 'Develop a "Weekly Discovery" feature that introduces users to new content aligned with their preferences.'
        })

    if 'unique_artists' in feature_names or 'unique_songs' in feature_names:
        recommendations.append({
            'category': 'Content Diversity',
            'title': 'Content Diversity Program',
            'description': 'Users who explore more diverse content tend to remain engaged longer. Encourage exploration of new artists and songs.',
            'implementation': 'Create curated playlists that gradually introduce users to new genres and artists similar to their current preferences.'
        })

    if 'is_paid' in feature_names:
        recommendations.append({
            'category': 'Subscription',
            'title': 'Subscription Tier Optimization',
            'description': 'Different patterns of churn exist between free and paid users. Tailor retention strategies to each segment.',
            'implementation': 'For free users, highlight premium features through targeted campaigns. For paid users, introduce loyalty rewards and exclusive content.'
        })

    if 'is_error_sum' in feature_names:
        recommendations.append({
            'category': 'Technical',
            'title': 'Error Reduction Initiative',
            'description': 'Technical errors correlate with increased churn rates. Prioritize fixing common errors and improve error handling.',
            'implementation': 'Develop a system to track and prioritize errors by their impact on user experience. Create more user-friendly error messages.'
        })

    if cluster_interpretations:
        high_risk_clusters = [c for c, interp in cluster_interpretations.items()
                             if 'High Risk' in interp['description']]
        if high_risk_clusters:
            high_risk = high_risk_clusters[0]
            interp = cluster_interpretations[high_risk]
            recommendations.append({
                'category': 'Targeted Intervention',
                'title': f'High-Risk Segment Focus: {interp["description"]}',
                'description': f'Cluster {high_risk} has a churn rate of {interp["churn_rate"]} and represents a critical segment for intervention.',
                'implementation': '; '.join(interp['recommendations'])
            })

    recommendations.append({
        'category': 'Retention',
        'title': 'Proactive Retention Program',
        'description': 'Implement a structured retention program targeting users before they show explicit signs of churning.',
        'implementation': 'Create automated journeys with targeted communications at key risk points (e.g., after 30 days of inactivity).'
    })

    print("\n----- Actionable Business Recommendations -----")
    with open("business_recommendations.txt", "w") as f:
        f.write("===== ACTIONABLE BUSINESS RECOMMENDATIONS =====\n\n")
        for i, rec in enumerate(recommendations, 1):
            print(f"\n{i}. [{rec['category']}] {rec['title']}")
            print(f"   {rec['description']}")
            print(f"   Implementation: {rec['implementation']}")
            f.write(f"{i}. [{rec['category']}] {rec['title']}\n")
            f.write(f"   {rec['description']}\n")
            f.write(f"   Implementation: {rec['implementation']}\n\n")
    return recommendations

In [14]:
def main():
    print("======================================================")
    print("   CUSTOMER CHURN PREDICTION AND ANALYSIS PROJECT     ")
    print("======================================================")
    print(f"Run date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    print("\nLoading data...")
    df = load_data('/content/drive/MyDrive/data.json')

    # New: Perform EDA
    df = perform_eda(df)

    # Preprocessing & Feature Engineering
    user_features = preprocess_data(df)

    # Clustering with detailed interpretation
    user_features = cluster_users(user_features)

    # Visualization
    visualize_data(user_features)

    # Prepare data for modeling
    all_features = [col for col in user_features.columns if col not in ['userId', 'churn']]
    target = 'churn'

    X = user_features[all_features]
    y = user_features[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

    # New: Hyperparameter tuning
    tuned_models = tune_hyperparameters(X_train, y_train)

    # Train models using tuned parameters
    trained_models = train_models(X_train, y_train, tuned_models=tuned_models)

    # Extract top features
    print("\nExtracting top features from models...")
    top_features = get_top_features(trained_models, all_features, top_n=10)
    print(f"Top 10 Features: {top_features}")

    # Retrain models using only top features
    print("\nRetraining models using only top 10 features...")
    trained_top = train_models(X_train[top_features], y_train, tuned_models=None)

    # Evaluate models
    print("\nFinal Evaluation on test set using top 10 features:")
    evaluation_results = {}
    for name, model in trained_top.items():
        print(f"\n{name}")
        evaluation_results[name] = evaluate_model(model, X_test[top_features], y_test)['roc_auc']

    # Plot ROC curves
    print("\nPlotting ROC Curves for all models:")
    auc_scores = plot_roc_curves(trained_top, X_test[top_features], y_test)

    # Generate business recommendations
    cluster_interpretations = None
    if 'cluster' in user_features.columns:
        # Extract cluster interpretations if available
        cluster_analysis = user_features.groupby('cluster').agg({
            'churn': 'mean',
            'total_listen': 'mean',
            'avg_listen': 'mean',
            'sessions': 'mean',
            'active_duration': 'mean',
            'membership_days': 'mean',
            'is_paid': 'mean',
            'unique_artists': 'mean',
            'unique_songs': 'mean',
            'avg_session_duration': 'mean',
            'userId': 'count'
        }).rename(columns={'userId': 'count'})

        cluster_interpretations, _ = interpret_clusters(cluster_analysis)

    generate_business_recommendations(trained_top, top_features, auc_scores, cluster_interpretations)

    print("\nAnalysis complete. Results saved to output files and plots.")

if __name__ == "__main__":
    main()


   CUSTOMER CHURN PREDICTION AND ANALYSIS PROJECT     
Run date: 2025-04-14 23:06:05

Loading data...

===== EXPLORATORY DATA ANALYSIS =====
Dataset shape: (26259199, 18)

Data types:
status             int64
gender            object
length           float64
firstName         object
level             object
lastName          object
registration     float64
userId            object
ts                 int64
auth              object
page              object
sessionId          int64
location          object
itemInSession      int64
userAgent         object
song              object
artist            object
method            object
dtype: object

Missing values per column:
status                 0
gender            778479
length           5408927
firstName         778479
level                  0
lastName          778479
registration      778479
userId                 0
ts                     0
auth                   0
page                   0
sessionId              0
location          778479